In [1]:
import tensorflow as tf
import cv2
import numpy as np
from enum import Enum
import time
import psutil
import mediapipe as mp
import math

2024-04-02 15:02:44.727800: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 15:02:45.071373: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 15:02:45.071487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 15:02:45.142723: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-02 15:02:45.279533: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 15:02:45.280504: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
thunder_interpreter = tf.lite.Interpreter(model_path='../models/movenet-lightning.tflite')
thunder_interpreter.allocate_tensors()

input_details = thunder_interpreter.get_input_details()
output_details = thunder_interpreter.get_output_details()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

I0000 00:00:1712050370.402376 1029907 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712050370.412853 1030219 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.0.4-0ubuntu1~22.04.1), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)


In [4]:
CONFIDENCE_THRESHOLD = 0.2

class Keypoint(Enum):
    NOSE = 0
    RIGHT_EYE = 2
    LEFT_EYE = 5
    RIGHT_EAR = 7
    LEFT_EAR = 8
    RIGHT_SHOULDER = 11
    LEFT_SHOULDER = 12
    RIGHT_ELBOW = 13
    LEFT_ELBOW = 14
    RIGHT_WRIST = 15
    LEFT_WRIST = 16
    RIGHT_HIP = 23
    LEFT_HIP = 24
    RIGHT_KNEE = 25
    LEFT_KNEE = 26
    RIGHT_ANKLE = 27
    LEFT_ANKLE = 28
    
    def __int__(self):
        return self.value
    
    @classmethod
    def all_keypoints(cls):
        return {key.name: key.value for key in cls}

class Edges(Enum):
    NOSE_TO_RIGHT_EYE = (Keypoint.NOSE, Keypoint.RIGHT_EYE, 'm')
    NOSE_TO_LEFT_EYE = (Keypoint.NOSE, Keypoint.LEFT_EYE, 'c')
    RIGHT_EYE_TO_RIGHT_EAR = (Keypoint.RIGHT_EYE, Keypoint.RIGHT_EAR, 'm')
    LEFT_EYE_TO_LEFT_EAR = (Keypoint.LEFT_EYE, Keypoint.LEFT_EAR, 'c')
    NOSE_TO_RIGHT_SHOULDER = (Keypoint.NOSE, Keypoint.RIGHT_SHOULDER, 'm')
    NOSE_TO_LEFT_SHOULDER = (Keypoint.NOSE, Keypoint.LEFT_SHOULDER, 'c')
    RIGHT_SHOULDER_TO_RIGHT_ELBOW = (Keypoint.RIGHT_SHOULDER, Keypoint.RIGHT_ELBOW, 'm')
    LEFT_SHOULDER_TO_LEFT_ELBOW = (Keypoint.LEFT_SHOULDER, Keypoint.LEFT_ELBOW, 'c')
    RIGHT_ELBOW_TO_RIGHT_WRIST = (Keypoint.RIGHT_ELBOW, Keypoint.RIGHT_WRIST, 'm')
    LEFT_ELBOW_TO_LEFT_WRIST = (Keypoint.LEFT_ELBOW, Keypoint.LEFT_WRIST, 'c')
    RIGHT_SHOULDER_TO_RIGHT_HIP = (Keypoint.RIGHT_SHOULDER, Keypoint.RIGHT_HIP, 'm')
    LEFT_SHOULDER_TO_LEFT_HIP = (Keypoint.LEFT_SHOULDER, Keypoint.LEFT_HIP, 'c')
    RIGHT_HIP_TO_RIGHT_KNEE = (Keypoint.RIGHT_HIP, Keypoint.RIGHT_KNEE, 'm')
    LEFT_HIP_TO_LEFT_KNEE = (Keypoint.LEFT_HIP, Keypoint.LEFT_KNEE, 'c')
    RIGHT_KNEE_TO_RIGHT_ANKLE = (Keypoint.RIGHT_KNEE, Keypoint.RIGHT_ANKLE, 'm')
    LEFT_KNEE_TO_LEFT_ANKLE = (Keypoint.LEFT_KNEE, Keypoint.LEFT_ANKLE, 'c')
    LEFT_SHOULDER_TO_RIGHT_SHOULDER = (Keypoint.LEFT_SHOULDER,Keypoint.RIGHT_SHOULDER, 'y')
    LEFT_HIP_TO_RIGHT_HIP = (Keypoint.LEFT_HIP,Keypoint.RIGHT_HIP, 'y')
    
    @classmethod
    def all_edges(cls):
        return [(edge.value[0].value, edge.value[1].value, edge.value[2]) for edge in cls]

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
def draw_keypoints(frame, keypoints, confidence_threshold=0):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold=0):
    y, x, _ = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    color_mapping = {'m': (255, 0, 255), 'c': (255, 255, 0), 'y': (0, 255, 255)}
    
    for p1,p2, color in edges:
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), color_mapping[color], 2 )

In [7]:
def calculate_angle(frame, keypoints, keypoint1, keypoint2, keypoint3, confidence_threshold=0.0):
    y, x, _ = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    keypoint1_coordinate = shaped[int(keypoint1)]
    keypoint2_coordinate = shaped[int(keypoint2)]
    keypoint3_coordinate = shaped[int(keypoint3)]
    
    if keypoint1_coordinate[2] < confidence_threshold or keypoint2_coordinate[2] < confidence_threshold or keypoint3_coordinate[2] < confidence_threshold:
        return None
    
    # Convert normalized coordinates to pixel coordinates
    y_scale, x_scale = 1.0, 1.0 
    kp1_x, kp1_y = keypoint1_coordinate[0] * x_scale, keypoint1_coordinate[1] * y_scale
    kp2_x, kp2_y = keypoint2_coordinate[0] * x_scale, keypoint2_coordinate[1] * y_scale
    kp3_x, kp3_y = keypoint3_coordinate[0] * x_scale, keypoint3_coordinate[1] * y_scale
    
    # Calculate vectors
    vector1 = np.array([kp1_x - kp2_x, kp1_y - kp2_y])
    vector2 = np.array([kp3_x - kp2_x, kp3_y - kp2_y])
    
    # Calculate dot product
    dot_product = np.dot(vector1, vector2)
    
    # Calculate magnitudes
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    
    # Calculate cosine of the angle
    cosine_angle = dot_product / (magnitude1 * magnitude2)
    
    # Calculate the angle in radians
    angle_rad = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    # Convert angle to degrees
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

In [11]:
def count_reps(exercise_type, frame, keypoints, confidence_threshold=0.0):
    y, x, _ = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    global rep
    global state
    
    if exercise_type == "curls":

        angle = calculate_angle(frame, keypoints,  Keypoint.LEFT_SHOULDER,
                                Keypoint.LEFT_ELBOW, Keypoint.LEFT_WRIST, confidence_threshold)
        
        if angle == None:
            return
        if angle < 30:
            state = "up"
        if angle > 140 and state == 'up':
            state = "down"
            rep += 1
        if angle > 140 and state == None:
            state = "down"

        shoulder_keypoint = shaped[int(Keypoint.LEFT_ELBOW)]
        kp_x, kp_y = int(shoulder_keypoint[0]), int(shoulder_keypoint[1])

        cv2.putText(frame, f'{angle:.2f}', (kp_x, kp_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    elif exercise_type == "squats":
        
        # Calculate knee angles
        left_knee_angle = calculate_angle(frame, keypoints,Keypoint.LEFT_HIP, Keypoint.LEFT_KNEE, Keypoint.LEFT_ANKLE)
        right_knee_angle = calculate_angle(frame, keypoints,Keypoint.RIGHT_HIP, Keypoint.RIGHT_KNEE, Keypoint.RIGHT_ANKLE)

        # Calculate hip angles
        left_hip_angle = calculate_angle(frame, keypoints,Keypoint.LEFT_SHOULDER, Keypoint.LEFT_HIP, Keypoint.LEFT_KNEE)
        right_hip_angle = calculate_angle(frame, keypoints,Keypoint.RIGHT_SHOULDER, Keypoint.RIGHT_HIP, Keypoint.RIGHT_KNEE)
        
        if(left_knee_angle == None or right_knee_angle == None or left_hip_angle == None or right_hip_angle == None):
            return
        
        thr = 135
        if (left_knee_angle < thr) and (right_knee_angle < thr) and (left_hip_angle < thr) and (right_hip_angle < thr):
            state = "down"
        if (left_knee_angle > thr) and (right_knee_angle > thr) and (left_hip_angle > thr) and (right_hip_angle > thr) and (state == 'down'):
            state = 'up'
            rep += 1
        if (left_knee_angle > thr) and (right_knee_angle > thr) and (left_hip_angle > thr) and (right_hip_angle > thr) and (state == None):
            state = 'up'

    elif exercise_type == "press":
        
        elbow_angle = calculate_angle(frame, keypoints,  Keypoint.LEFT_SHOULDER,
                                Keypoint.LEFT_ELBOW, Keypoint.LEFT_WRIST, confidence_threshold)
        
        # Compute distances between joints
        shoulder2elbow_dist = abs(math.dist(shaped[int(Keypoint.LEFT_SHOULDER)],shaped[int(Keypoint.LEFT_ELBOW)]))
        shoulder2wrist_dist = abs(math.dist(shaped[int(Keypoint.LEFT_SHOULDER)],shaped[int(Keypoint.LEFT_WRIST)]))
        
        if (elbow_angle > 130) and (shoulder2elbow_dist < shoulder2wrist_dist):
            state = "up"
        if (elbow_angle < 50) and (shoulder2elbow_dist > shoulder2wrist_dist) and (state =='up'):
            state='down'
            rep += 1
        if (elbow_angle < 50) and (shoulder2elbow_dist > shoulder2wrist_dist) and (state == None):
            state='down'

    else:
        pass

In [18]:
# cap = cv2.VideoCapture('../dataset/hammer curl/hammer curl_19.mp4')
cap = cv2.VideoCapture('../dataset/shoulder press/shoulder press_12.mp4')
# cap = cv2.VideoCapture('../dataset/barbell biceps curl/barbell biceps curl_46.mp4')

# cap = cv2.VideoCapture(0)

rep = 0
state = None

prev_frame_time = 0
new_frame_time = 0

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret: 
      break
    
    # Preprocess frame
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    
    if results.pose_landmarks:
         mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    else: continue
    
    keypoints_with_scores = [(landmark.x,
                              landmark.y,
                              landmark.visibility)
                             for landmark in results. pose_landmarks.landmark]
    
    # Make predictions 
    # thunder_interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    # thunder_interpreter.invoke()
    # keypoints_with_scores = thunder_interpreter.get_tensor(output_details[0]['index'])

    # CPU analysis
    new_frame_time = time.time() 
    
    fps = 1/(new_frame_time-prev_frame_time) 
    prev_frame_time = new_frame_time 
    fps = str(int(fps)) 
    
    ram_usage = psutil.virtual_memory().percent
    cpu_usage = psutil.cpu_percent()

    # putting the FPS count on the frame 
    cv2.putText(frame, 'FPS: {}'.format(fps), (1000, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (100, 255, 0), 2) 
    
    # Display the RAM usage
    cv2.putText(frame, f"RAM Usage: {ram_usage}%", (1000, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (100, 255, 0), 2)
    
    # Display CPU usage
    cv2.putText(frame, f"CPU Usage: {cpu_usage}%", (1000, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (100,255,0), 2)

    
    # Rendering     
    # draw_connections(frame, keypoints_with_scores, Edges.all_edges(), CONFIDENCE_THRESHOLD)
    # draw_keypoints(frame, keypoints_with_scores, CONFIDENCE_THRESHOLD)
    
    count_reps('press', frame, keypoints_with_scores, CONFIDENCE_THRESHOLD)
    
    cv2.rectangle(frame, (0,0), (360, 40), 0.5, -1)
    cv2.putText(frame, 'reps ' + str(rep), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, ' - state: ' + str(state), (110,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.imshow('Output Video', frame)

    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()